In [23]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import pandas as pd
import cv2
import numpy as np
import joblib

# Leer LBP_Codes desde el archivo Excel
LBP_Codes = pd.read_excel(r'G:\Mi unidad\Semestre 8\ReconocimientoDePatrones\Reto rostros\LBP_Codes.xlsx')
LBP_Codes = np.array(LBP_Codes)

# Función para procesar la imagen con LBP
def LBP_Brinez(Img_gris, LBP_Codes):
    [Fl, Cl] = Img_gris.shape
    Matriz_LBP = np.zeros((Fl, Cl))
    Pesos = np.array([[1, 2, 4], [128, 0, 8], [64, 32, 16]])

    for j in range(1, Cl - 1, 1):
        for i in range(1, Fl - 1, 1):
            Region = Img_gris[i - 1:i + 2, j - 1:j + 2]
            Referencia = Img_gris[i, j]
            Region_Bin = Region >= Referencia
            Escalada = Region_Bin * Pesos
            Suma = np.sum(Escalada)
            Codigo = LBP_Codes[Suma, 1]
            Matriz_LBP[i, j] = Codigo

    LBP_Histograma = np.histogram(Matriz_LBP, bins=59, range=(0, 58))
    LBP_Histograma = LBP_Histograma[0]
    LBP_Vector = np.zeros((1, len(LBP_Histograma)))
    for i in range(len(LBP_Histograma)):
        LBP_Vector[0, i] = LBP_Histograma[i]
    return (Matriz_LBP, LBP_Vector)

def cargar_imagen_y_procesar():
    global root, imagen_tk, nombre_usuario, descuento, mensaje_descuento
    archivo_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
    if archivo_path:
        Imagen2 = cv2.imread(archivo_path)
        Filas = 800
        Columnas = 800
        I_gris = cv2.cvtColor(Imagen2, cv2.COLOR_BGR2GRAY)
        I_gris = cv2.resize(I_gris, (Filas, Columnas), interpolation=cv2.INTER_AREA)

        # Cargar el modelo desde el archivo
        Modelo_entrenado = joblib.load('G:\Mi unidad\Semestre 8\ReconocimientoDePatrones\Reto rostros\Modelo_Rostros2_KNN_JN.pkl')

        # Aplicar el proceso a la imagen de validación
        Matriz_Características = np.zeros((1, 59 * 16))
        Detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        try:
            Cara = Detector.detectMultiScale(I_gris, scaleFactor=1.1, minNeighbors=10, minSize=(20, 20), maxSize=(300, 300))
            Tamaño_Roi = int(48)
            Filas_2 = Tamaño_Roi
            Columnas_2 = Tamaño_Roi
            Particiones = Filas_2 / 4
            for (x, y, w, h) in Cara:
                Recorte = I_gris[y:y + h, x:x + w]
            del Cara
            Recorte = cv2.resize(Recorte, (Filas_2, Columnas_2))
            Contador = 0
            for j in range(0, 4):
                for k in range(0, 4):
                    Matriz_LBP, Matriz_Características[0, Contador * 59:(Contador * 59 + 59)] = LBP_Brinez(
                        Recorte[j * 12:(j * 12 + 12), k * 12:(k * 12 + 12)], LBP_Codes)
                    Contador += 1

            Prediccion = Modelo_entrenado.predict(Matriz_Características)

            Nombres = ["Yeicy", "Gerson", "Nora", "Francisco", "Lina", "Cata", "walter"]
            precio_base = 10  # Precio base del menú
            if int(Prediccion[0]) - 1 in range(len(Nombres)):
                nombre_usuario = Nombres[int(Prediccion[0]) - 1]
                mensaje_bienvenida = f"Bienvenido, {nombre_usuario}!"
                label_bienvenida.config(text=mensaje_bienvenida)
                descuento = 20
                
                # Mostrar la imagen cargada en la ventana principal
                imagen = Image.open(archivo_path)
                imagen = imagen.resize((300, 300))
                imagen_tk = ImageTk.PhotoImage(imagen)
                label_imagen_principal.config(image=imagen_tk)

                # Solicitar al usuario que elija un menú
                #mostrar_ventana_menu()
                
                boton_elegirmenu = tk.Button(root, text="Elegir menú", command=lambda: mostrar_ventana_menu())
                boton_elegirmenu.pack(pady=5)

            else:
                mensaje_bienvenida = f"Usuario no registrado."
                nombre_usuario = ""
                label_bienvenida.config(text=mensaje_bienvenida)
                descuento = 20
                
                # Mostrar la imagen cargada en la ventana principal
                imagen = Image.open(archivo_path)
                imagen = imagen.resize((300, 300))
                imagen_tk = ImageTk.PhotoImage(imagen)
                label_imagen_principal.config(image=imagen_tk)

                # Solicitar al usuario que elija un menú
                #mostrar_ventana_menu()
                
                boton_elegirmenu = tk.Button(root, text="Elegir menú", command=lambda: mostrar_ventana_menu())
                boton_elegirmenu.pack(pady=5)
                
        except:
            #print("No se detectaron caras en la imagen.")
            #messagebox.showwarning("Error", "No se detectaron caras en la imagen.")
            mensaje_bienvenida = f"Usuario no registrado."
            nombre_usuario = ""
            label_bienvenida.config(text=mensaje_bienvenida)
            descuento = 0
            
            # Mostrar la imagen cargada en la ventana principal
            imagen = Image.open(archivo_path)
            imagen = imagen.resize((300, 300))
            imagen_tk = ImageTk.PhotoImage(imagen)
            label_imagen_principal.config(image=imagen_tk)

            # Solicitar al usuario que elija un menú
            #mostrar_ventana_menu()
            boton_elegirmenu = tk.Button(root, text="Elegir menú", command=lambda: mostrar_ventana_menu())
            boton_elegirmenu.pack(pady=5)
   

def mostrar_ventana_menu():
    global root
    ventana_menu = tk.Toplevel(root)
    ventana_menu.title("Seleccionar Menú")

    mensaje_descuento = f"Señor(a) {nombre_usuario} tienes un {descuento}% de descuento, puedes escoger tu menú."
    etiqueta_resumen = tk.Label(ventana_menu, text=mensaje_descuento)
    etiqueta_resumen.pack(padx=20, pady=10)

    # Crear botones para el menú
    boton_menu1 = tk.Button(ventana_menu, text="Menú 1 (Bandeja paisa)", command=lambda: mostrar_resumen_menu("Bandeja paisa"))
    boton_menu1.pack(pady=5)

    boton_menu2 = tk.Button(ventana_menu, text="Menú 2 (Ajiaco)", command=lambda: mostrar_resumen_menu("Ajiaco"))
    boton_menu2.pack(pady=5)

    boton_menu3 = tk.Button(ventana_menu, text="Menú 3 (Pescado)", command=lambda: mostrar_resumen_menu("Pescado"))
    boton_menu3.pack(pady=5)

def mostrar_resumen_menu(menu):
    precioDescuento = 15000 * (1-descuento/100)
    ventana_resumen = tk.Toplevel(root)
    ventana_resumen.title("Resumen del Menú")
    mensaje = f"Señor(a) {nombre_usuario}, la selección de su menú fue exitosa.\nHa elegido {menu}:\n el precio a pagar es de: {precioDescuento}"
    etiqueta_resumen = tk.Label(ventana_resumen, text=mensaje)
    etiqueta_resumen.pack(padx=20, pady=10)
    
    boton_cambio = tk.Button(ventana_resumen, text="Cambiar menú", command=ventana_resumen.destroy)
    boton_cambio.pack(side=tk.LEFT, padx=10, pady=10)
    
    boton_cerrar = tk.Button(ventana_resumen, text="Cerrar", command=root.destroy)
    boton_cerrar.pack(side=tk.RIGHT, padx=10, pady=10)

# Crear la ventana principal
root = tk.Tk()
root.title("Sistema de Reconocimiento de Usuario")
root.geometry("400x500")

# Botón para cargar la imagen y procesarla
boton_cargar = tk.Button(root, text="Cargar Imagen", command=cargar_imagen_y_procesar)
boton_cargar.pack(pady=20)

# Crear un widget Label para mostrar la imagen seleccionada
imagen_tk = None  # Inicializar imagen_tk
label_imagen_principal = tk.Label(root)
label_imagen_principal.pack()

# Crear un widget Label para mostrar el mensaje de bienvenida
label_bienvenida = tk.Label(root, text="")
label_bienvenida.pack(pady=10)


# Iniciar el bucle de la aplicación
root.mainloop()
